This is a script for **adding coordinates and project IDs** (e.g. person IDs) to a factoid table in the DigiKAR project. The geocoding is not performed from scratch; instead, coordinates are added from a previously geocoded table of place names.

The package mainly uses the Pandas package in Python to read and manipulate EXCEL data as DataFrames. DataFrames are 2-dimensional data representations in rows and columns. They can be written to different file formats such as CSV, EXCEL, JSON or RDF.

First of all, we need to connect this Colab notebook with your Google Drive and define the directory for input and output data.


In [ ]:
## mount drive
from google.colab import drive
drive.mount("/content/drive")
directory="/content/drive/My Drive/Colab_DigiKAR/"

In the second step, we have to install additional Packages needed for working with CSV, EXCEL and DataFrames.

In [ ]:
## install packages that are not part of Python's standard distribution

!pip install xlsxwriter
!pip install pandas
!pip install numpy

In **step 1**, we can import the packages to the script and load our data. Before merging the input files, names will be normalised as some have access spaces, capitalised surnames, or inverted first and last names.

The combined data will be written to a new dataframe and displayed.

In [ ]:
import xlsxwriter
import csv
import pandas as pd
from pandas import DataFrame
import numpy as np
import os
import re

# path to input files

factoid_paths=["https://github.com/ieg-dhr/DigiKAR/raw/main/Sample%20Data/Factoid-PROFS-consolidation-STEP3-events-split-v10.xlsx"]

# define dataframe for final output

f_to_add=[]

# structure of input files

# obligatory columns in valid factoid list

column_names = ["factoid_ID",
                "pers_ID",
                "alternative_names",
                "event_type",
                "event_after-date",
                "event_before-date",
                "event_start",
                "event_end",
                "event_date",
                "pers_title",
                "pers_function",
                "place_name",
                "inst_name",
                "rel_pers",
                "source_quotations",
                "additional_info",
                "comment",
                "info_dump",
                "source",
                "source_site"]

frame_list=[]
for file in factoid_paths:
    df = pd.read_excel(file, index_col=None, dtype=str) # axis=1, sort=False sheet_name='FactoidList'
    df = df.fillna("n/a") # replace empty fields for string
    frame_list.append(df)

f_unique = pd.concat(frame_list, axis=0, ignore_index=True, sort=False)

print("There are ", len(f), "items in your DataFrame!")

# delete all duplicate rows with exact matches

display(f_unique)


In **step 2**, we add the person ID's from the person ontology list and also add coordinates from Geonames and Google.




In [ ]:
# Merge input dataframe with dfs containing person IDs and geocoding
# documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

## Read person IDs from Github
## columns: pers_ID_MB, pers_name, alternative_names, Unnamed: 4, name_new_fs, pers_ID_FS,poi, pers_name_corr, freq
infile1="https://github.com/ieg-dhr/DigiKAR/raw/main/OntologyFiles/Factoid_PersonNames_merged.xlsx" # has to contain pers_name column!
person_df = pd.read_excel(infile1)

## Read geocoding from Github
infile2="https://github.com/ieg-dhr/DigiKAR/raw/main/OntologyFiles/Ortsontologie_Geocoded_gepr%C3%BCft.xlsx" # has to contain place_name column!
geo_df = pd.read_excel(infile2)

## merge dataframes horizontally

merged_df1 = pd.merge(f_unique, geo_df, on='place_name', how='inner').fillna("n/a")
merged_df2 = pd.merge(merged_df1, person_df, on=('pers_name', "alternative_names"), how="inner").fillna("n/a")

display(merged_df2)

**Step 3** adds event values from the event value Python dictionary on Github.




In [ ]:
## load external dictionary with EVENT VALUES
# following method 2 on https://www.geeksforgeeks.org/how-to-read-dictionary-from-file-in-python/

# importing the module
import requests
import ast

master = "https://raw.githubusercontent.com/ieg-dhr/DigiKAR/main/Data%20Categorisation/Event_value_dict.txt" # add Sven's new mapping
req = requests.get(master)
req = req.text
print(req)

# reconstructing the data as a dictionary
event_value_dict = ast.literal_eval(req)
print(type(event_value_dict))

# add event values from dict to data frame

try:
    test = event_value_dict["Aufschwörung"] # random test if valid dict
    print("Value for chosen key: ", test)
except:
    print("Invalid dict structure!")

merged_df2['event_value'] = merged_df2['event_type'].map(event_value_dict) # optional: na_action='ignore'

display(merged_df2)

**Step 4** is to write the results to a single output file.

In [ ]:
# write all results to new EXCEL file

workbook=directory+'FACTOIDS_consolidated/Factoid_PROFS_v10_geocoded-with-IDs.xlsx'
print(workbook)
writer = pd.ExcelWriter(workbook, engine='xlsxwriter') # create a Pandas Excel writer using XlsxWriter as the engine.
merged_df2.to_excel(writer, sheet_name='FactCons') # Convert the dataframe to an XlsxWriter Excel object.
writer.save() # Close the Pandas Excel writer and output the Excel file.
print("Done.")

Check the output files and repeat process if necessary.

Script by Monika Barget, Maastricht/Mainz

June 2023
